In [1]:
import numpy as np 
import matplotlib.pyplot as plt
import stumpy
import pandas as pd
from scipy import signal
from sklearn.linear_model import Ridge
from scipy.spatial import distance
from fatf.utils.kernels import exponential_kernel 

import warnings
warnings.filterwarnings('ignore')

import sys
sys.path.append('../')

from Utils.explanations import LIMESegment, NEVES, LEFTIST, NNSegment, RBP, background_perturb
from  Utils.data import loadUCRDataID
from  Utils.models import *
from  Utils.metrics import *

In [2]:
# dataset_map = [('Coffee', 0),
#                 ('Strawberry', 1),
#                    ('GunPointOldVersusYoung', 2),
#                    ('HandOutlines', 3),
#                     ('yoga', 4),
#                     ('ECG200', 5),
#                     ('GunPointMaleVersusFemale', 6),
#                     ('DodgerLoopGame', 7),
#                     ('Chinatown', 8),
#                     ('FreezerSmallTrain', 9),
#                     ('HouseTwenty', 10),
#                     ('WormsTwoClass', 11)
#                     ]

dataset_map = [('ECG200', 0)]

In [3]:
datasets = {}
for data_idx in dataset_map:
    datasets[data_idx[0]] = loadUCRDataID(data_idx[1])
    # print(loadUCRDataID(data_idx[1]))

Loading train / test dataset :  ../Data/Coffee_TRAIN ../Data/Coffee_TEST


In [4]:
"""Datasets['Coffee'] : Train x, Train y, Test x, Test y

Train x : N x T x 1
Train y: N x 1 
Test x: N' x T x 1
Test y: N' x 2"""

"Datasets['Coffee'] : Train x, Train y, Test x, Test y\n\nTrain x : N x T x 1\nTrain y: N x 1 \nTest x: N' x T x 1\nTest y: N' x 2"

In [4]:
for data_idx in datasets.keys():
    print(datasets[data_idx][0].shape)

(28, 286, 1)


In [5]:
BATCH_SIZES = [8, 64, 32, 64, 64, 16, 16, 4, 4, 4, 8, 16]
WINDOW_SIZES = [20, 20, 10, 100, 50, 10, 10, 20, 3, 10, 200, 100]
CPS = [5, 5, 4, 8, 5, 3, 4, 5, 2, 5, 8, 6]

In [6]:
models = ['knn','cnn','lstmfcn']
trained_models = {}
i = 0
for data_idx in datasets.keys():
    print(data_idx)
    trained_models[data_idx] = {}
    trained_models[data_idx]['knn'] = train_KNN_model(datasets[data_idx][0],datasets[data_idx][1])
    model_cnn = make_CNN_model(datasets[data_idx][0].shape[1:])
    trained_models[data_idx]['cnn'] = train_CNN_model(model_cnn,
                                                      datasets[data_idx][0],
                                                      datasets[data_idx][1],
                                                      epochs=2,
                                                      batch_size=BATCH_SIZES[i])[0]
    model_lstmfcn = make_LSTMFCN_model(datasets[data_idx][0].shape[1])
    trained_models[data_idx]['lstmfcn'] = train_LSTMFCN_model(model_lstmfcn,
                                                      datasets[data_idx][0],
                                                      datasets[data_idx][1],
                                                      datasets[data_idx][2],
                                                      datasets[data_idx][3],
                                                      epochs=2,
                                                      batch_size=BATCH_SIZES[i])
    i = i + 1

ECG200
Epoch 1/2
3/3 [==============================] - 1s 83ms/step - loss: 0.6719 - sparse_categorical_accuracy: 0.6364 - val_loss: 0.7288 - val_sparse_categorical_accuracy: 0.0000e+00
Epoch 2/2
3/3 [==============================] - 0s 14ms/step - loss: 0.6301 - sparse_categorical_accuracy: 0.6364 - val_loss: 0.7660 - val_sparse_categorical_accuracy: 0.0000e+00
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 286, 1)]     0                                            
__________________________________________________________________________________________________
permute (Permute)               (None, 1, 286)       0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1d_3 (Conv1D)     

In [8]:
"""model_KNN shape : N x T 
## model_CNN/LSTMFCN shape  : N x T x 1


SyntaxError: EOF while scanning triple-quoted string literal (3663073547.py, line 2)

In [15]:
# MODEL_TYPES = ['class','proba','proba']
MODEL_TYPES = ['class']

dict_keys(['knn', 'cnn', 'lstmfcn'])

In [9]:
def reshaper(x,j):
    if j == 0:
        return x.reshape(x.shape[0])
    else:
        return x


In [16]:
explanations = {}
i = 0 
noisy_explanations = {} # For Robustness later
for data_idx in datasets.keys():
    print('processing explanations for: ' + str(data_idx) + '\n')
    explanations[data_idx] = {}
    noisy_explanations[data_idx] = {}
    j = 0
    for model_idx in trained_models[data_idx].keys():
        print('processing explanations for: ' + str(model_idx) + '\n')
        explanations[data_idx][model_idx] = {}
        noisy_explanations[data_idx][model_idx] = {}
        explanation_set = datasets[data_idx][2][0:2]
        explanations[data_idx][model_idx]['LS'] = [LIMESegment(reshaper(x,j), trained_models[data_idx][model_idx], model_type=MODEL_TYPES[j], window_size=WINDOW_SIZES[i], cp=CPS[i]) for x in explanation_set]
        
        # # explanations[data_idx][model_idx]['N'] = [NEVES(x, trained_models[data_idx][model_idx], datasets[data_idx][0], model_type=MODEL_TYPES[j]) for x in explanation_set]
        # explanations[data_idx][model_idx]['LF'] = [LEFTIST(reshaper(x,j), trained_models[data_idx][model_idx], datasets[data_idx][0], model_type=MODEL_TYPES[j],) for x in explanation_set]
        

        # noisy_set = np.asarray([add_noise(x) for x in explanation_set])
        
        # noisy_explanations[data_idx][model_idx]['LS'] = [LIMESegment(reshaper(x,j), trained_models[data_idx][model_idx], model_type=MODEL_TYPES[j], window_size=WINDOW_SIZES[i], cp=CPS[i]) for x in noisy_set]
        # # noisy_explanations[data_idx][model_idx]['N'] = [NEVES(x, trained_models[data_idx][model_idx], datasets[data_idx][0], model_type=MODEL_TYPES[j]) for x in noisy_set]
        # noisy_explanations[data_idx][model_idx]['LF'] = [LEFTIST(reshaper(x,j), trained_models[data_idx][model_idx], datasets[data_idx][0], model_type=MODEL_TYPES[j],) for x in noisy_set]
        
        # j = j + 1
    # print(explanations[data_idx]['knn']['LS'])
    i = i + 1

processing explanations for: ECG200

processing explanations for: knn

processing explanations for: cnn

processing explanations for: lstmfcn

[(array([-0.04077734, -0.03199378, -0.25846592, -0.09899664,  0.26749658,
       -0.04891599]), [0, 37, 142, 174, 195, 242, -1]), (array([0., 0., 0., 0., 0., 0.]), [0, 141, 174, 193, 242, 259, -1])]


In [29]:
print(explanations[data_idx]['cnn']['LS'][0])

(array([[ 4.37788806e-04,  1.67927591e-03,  7.06213124e-04,
        -5.28547711e-05,  1.28932976e-03, -1.64331453e-05],
       [-4.61302202e-03,  3.15371341e-04,  1.14283047e-03,
         5.00663670e-04,  1.75474391e-03, -1.56491291e-05]]), [0, 37, 142, 174, 195, 242, -1])


In [36]:
trained_models['ECG200']['cnn'].predict(datasets[data_idx][2][1])
# datasets.keys()

array([[0.51120716, 0.5051165 ],
       [0.5107265 , 0.5052757 ],
       [0.51076806, 0.50526196],
       [0.51148444, 0.5050385 ],
       [0.51216847, 0.504884  ],
       [0.51287836, 0.50488067],
       [0.5134346 , 0.5048947 ],
       [0.51347107, 0.50489557],
       [0.51350135, 0.50489634],
       [0.5136781 , 0.50489837],
       [0.5138141 , 0.50489783],
       [0.513836  , 0.5048978 ],
       [0.5138741 , 0.5048976 ],
       [0.51394254, 0.50489736],
       [0.51396793, 0.50489724],
       [0.5140649 , 0.5048969 ],
       [0.51408726, 0.5048968 ],
       [0.5140526 , 0.50489694],
       [0.5140202 , 0.50489706],
       [0.51395386, 0.5048973 ],
       [0.51404494, 0.504897  ],
       [0.5141131 , 0.5048967 ],
       [0.5141523 , 0.5048966 ],
       [0.5143198 , 0.50490737],
       [0.51437336, 0.504914  ],
       [0.5143109 , 0.5049063 ],
       [0.51427627, 0.504902  ],
       [0.51433176, 0.50490886],
       [0.5144494 , 0.5049234 ],
       [0.51461273, 0.5049223 ],
       [0.

In [31]:
evaluation_metrics = {}
for data_idx in datasets.keys():
    evaluation_metrics[data_idx] = {}
    j = 0 
    for model_idx in trained_models[data_idx].keys():
        evaluation_metrics[data_idx][model_idx] = {}
        for explanation_idx in explanations[data_idx][model_idx].keys():
            evaluation_metrics[data_idx][model_idx][explanation_idx] = {}
            # Robustness
            evaluation_metrics[data_idx][model_idx][explanation_idx]['Robustness'] = robustness(explanations[data_idx][model_idx][explanation_idx],
                                                                                         noisy_explanations[data_idx][model_idx][explanation_idx])
            explanation_set = datasets[data_idx][2][0:2]
            explanation_labels = datasets[data_idx][3][0:2]
            if j == 0:
                explanation_predictions = trained_models[data_idx][model_idx].predict(explanation_set.reshape(explanation_set.shape[:2]))
            
            else:
                explanation_predictions = trained_models[data_idx][model_idx].predict(explanation_set)
                
                # Faithfulness
            evaluation_metrics[data_idx][model_idx][explanation_idx]['Faithfulness'] = faithfulness(explanations[data_idx][model_idx][explanation_idx],explanation_set,explanation_labels,explanation_predictions,trained_models[data_idx][model_idx],model_type=MODEL_TYPES[j])
        j+=1
    

[(array([ 0.15642081, -0.04147744, -0.2829171 , -0.04823439,  0.34049784,
        0.0258209 ]), [0, 37, 142, 174, 195, 242, -1]), (array([0., 0., 0., 0., 0., 0.]), [0, 141, 174, 193, 242, 259, -1])]
[(array([ 0.15642081, -0.04147744, -0.2829171 , -0.04823439,  0.34049784,
        0.0258209 ]), [0, 37, 142, 174, 195, 242, -1]), (array([0., 0., 0., 0., 0., 0.]), [0, 141, 174, 193, 242, 259, -1])]
[(array([-0.04649363, -0.08004188, -0.02418995,  0.04415766,  0.0242685 ,
       -0.07512611, -0.07568266,  0.0609746 , -0.0672467 , -0.00990373,
        0.03806916]), [0, 28, 56, 84, 112, 140, 168, 196, 224, 252, 280]), (array([-0.16573141, -0.00049511, -0.03258302, -0.0578341 , -0.13982137,
       -0.13128767, -0.0264127 , -0.07383631, -0.04282645,  0.00192508,
        0.05782108]), [0, 28, 56, 84, 112, 140, 168, 196, 224, 252, 280])]
[(array([-0.04649363, -0.08004188, -0.02418995,  0.04415766,  0.0242685 ,
       -0.07512611, -0.07568266,  0.0609746 , -0.0672467 , -0.00990373,
        0.03806